# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных   
    
* `children` — количество детей в семье   
* `days_employed` — общий трудовой стаж в днях   
* `dob_years` — возраст клиента в годах   
* `education` — уровень образования клиента    
* `education_id` — идентификатор уровня образования    
* `family_status` — семейное положение    
* `family_status_id` — идентификатор семейного положения   
* `gender` — пол клиента    
* `income_type` — тип занятости   
* `debt` — имел ли задолженность по возврату кредитов   
* `total_income` — ежемесячный доход   
* `purpose` — цель получения кредита    

## Предобработка данных

In [5]:
import pandas as pd
df = pd.read_csv('/Users/polina_lunyova/Documents/It/projects/final_version/2_Предобработка_данных /data.csv')
display(df.head())
display(df.info())
display(df.describe())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**   
В таблице имеются необработанные данные и ошибки. Прежде чем приступать к проверке гипотез, необходимо привести данные к нормальному виду.

### Обработка пропусков

In [6]:
# смотрим наличие пропусков 
df.isna().sum()

# заполняем медианным значением пропуски в ежемесячном доходе(NaN)
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

# заполняем медианным значением пропуски в таблице с трудовым стажем
df['days_employed'] = df['days_employed'].fillna(df.groupby('dob_years')['days_employed'].transform('median'))
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**   
В таблице обнаружены пропуски(NaN) в трудовом стаже и ежемесячном доходе. Возможно клиент предоставил не все данные. 
Пропуски заполнены медианным способом, т.к. удаление пропущенных строк приведет к искажению результатов.

### Замена типа данных

In [7]:
df.info()

# для изменения типа данных используем метод astype() т.к. с помощью этого метода можно перевести числа в целочисленные значение
df['total_income'] = df['total_income'].round().astype('int64')
df['days_employed'] = df['days_employed'].round().astype('int64')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**   
Совершена замена типов данных таблицы ежемесячного дохода (total_income) и общего трудового стажа (days_employed) из вещественного в целочисленный.

### Обработка дубликатов

In [8]:
df.head()
df.duplicated().sum()

# приведем все записи в стодбце "education" к нижнему регистру
df['education'] = df['education'].str.lower()
df['education'].value_counts()

# смотрим уникальные значения по каждому столбцу
for column in df:
    display(df[column].value_counts())
    
# исправляем ошибки в столбце 'children'
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()

df = df.drop_duplicates()
df.duplicated().sum()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

-1421      111
-1675       72
-1602       71
-1595       71
-1296       70
          ... 
-11220       1
 360343      1
 355300      1
 394233      1
 376840      1
Name: days_employed, Length: 9068, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

...

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64

142594    1105
172358     509
118514     414
150448     147
163082       4
          ... 
402526       1
230490       1
222294       1
160852       1
89635        1
Name: total_income, Length: 18591, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

0

**Вывод**   
В данной таблице были найдены дубликаты методом duplicated().sum() и удалены с помошью drop_duplicates(). 
Исправлены ошибки в столбце с  количеством детей в семье (children) при помощи замены методом replace().
Таблица с уровнем образования клиента (education) приведена к нижниму регистру.
В столбце gender присутствует значение 'XNA', но его можно не изменять т.к. данный стобец в дальнейших расчетах использоваться не будет.
Также в dob_years есть возраст '0', но этот столбец в расчетах также использоваться не будет.

## Лемматизация

In [9]:
#леммитизируем столбец целей получения кредита
from pymystem3 import Mystem
m = Mystem()
def lemmatization(row):
    lemmas = m.lemmatize(row['purpose'])
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
df['lemma_purpose'] = df.apply(lemmatization, axis=1)

#проверка получившихся столбцов с помощью леммитизации
print(df['lemma_purpose'].value_counts())

#проверяем совпадает ли сумма получившихся строк с изначальной
df['lemma_purpose'].value_counts().sum()
len(df['purpose'])

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lemma_purpose, dtype: int64


21454

**Вывод**  
Для облегчения дальнейших расчетов была применена лемматизация и объединены цели получения кредита, записанные в разной форме, в общие строки.

## Категоризация данных

In [10]:
#создадим функцию с нужными параметрами для объединения данных
def children_group(child):
    if child == 0:
        return 'без детей'
    if 1 <= child <= 3:
        return 'есть дети'
    return 'многодетные'

#создадим отдельный столбец с категориями 
df['children'] = df['children'].apply(children_group)

#разделим данные о доходе на равные группы с помощью qcut()
bin_labels_3 = ['низкий доход', 'средний доход', 'высокий доход']
df['total_group'] = pd.qcut(df['total_income'], q=[0, 1/3, 2/3, 1], labels=bin_labels_3)
df['total_group'].value_counts()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma_purpose,total_group
0,есть дети,-8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,недвижимость,высокий доход
1,есть дети,-4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,низкий доход
2,без детей,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,недвижимость,средний доход
3,есть дети,-4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование,высокий доход
4,без детей,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний доход


В целях наилучшего восприятия информации, данные были сгруппированы с помощью категоризации. Для разделения данных на равные группы был использован метод qcut(). Низкий доход клиента состовляет от 119261 до 172358, средний доход от 20667 до 119261, высокий доход от 172358 до 2265604 т.р. в месяц.

## Проверка зависимости между наличием детей и возвратом кредита в срок

In [11]:
pivot_table_children = df.pivot_table(index='children', columns='debt', values='days_employed', aggfunc='count')
pivot_table_children['percent_1'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0]) * 100
pivot_table_children

debt,0,1,percent_1
children,,,
без детей,13028,1063,7.543822
есть дети,6639,674,9.216464
многодетные,46,4,8.000000


**Вывод**   
Процент клиентов, у котрых есть дети, наиболее подвержены к задолженности по возврату кредита.

## Проверка зависимости между семейным положением и возвратом кредита в срок

In [12]:
pivot_table_family_status = df.pivot_table(index='family_status', columns='debt', values='days_employed', aggfunc='count')
pivot_table_family_status['percent_2'] = pivot_table_family_status[1] / (pivot_table_family_status[1] + pivot_table_family_status[0]) * 100
pivot_table_family_status

debt,0,1,percent_2
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


**Вывод**    
Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.

## Проверка зависимости между уровнем дохода и возвратом кредита в срок

In [13]:
pivot_table_income = df.pivot_table(index='total_group', columns='debt', values='days_employed', aggfunc='count')
pivot_table_income['percent_3'] = pivot_table_income[1] / (pivot_table_income[1] + pivot_table_income[0]) * 100
pivot_table_income

debt,0,1,percent_3
total_group,,,
низкий доход,6571,581,8.123602
средний доход,6653,632,8.675360
высокий доход,6489,528,7.524583


**Вывод**

Посмотрев на таблицу, можно увидеть, что клиенты с низкой ( от 119261 до 172358 т.р.) и средней (от 20667 до 119261 т.р.) доходностью имеют наибольшую вероятность просрочить выплаты по кредиту. 

## Проверка влияния разных целей кредита на его возврат в срок


In [14]:
pivot_table_purpose = df.pivot_table(index='lemma_purpose', columns='debt', values='days_employed', aggfunc='count')
pivot_table_purpose['percent_4'] = pivot_table_purpose[1] / (pivot_table_purpose[1] + pivot_table_purpose[0]) * 100
pivot_table_purpose

debt,0,1,percent_4
lemma_purpose,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**   
Клиенты, которые берут кредиты на автомобиль или образование, наиболее подвержены к задолженности по возврату кредита. 

## Общий вывод

В ходе исследования был произведен анализ данных платёжеспособности клиентов, а также определено влияние семейного положения и количество детей клиента на факт погашения кредита в срок.    
Были определены статусы клиента, наличие которых отрицательно влияет на платёжеспособность клиентов:
1. Клиент имеет от одного до трех детей
2. Клиент состоит в гражданском браке или не женат (не замужем)
3. Клиент имеет средний и низкий доходы
4. Клиент получает кредит на образование или автомобиль.